# Create Slack Procedures for SQL Server

## Adding the Token for slack into a Table

The following table holds the slack token, It would be best if the token is stored in an encrypted manner and decrypted when called by the application. As this is example code this is not done in this example. additionally the stored procedure would also be signed so that it can be used without needing elevated permissions to be accessed.

In [ ]:
Use master

CREATE TABLE SlackParams
(
    Token_name NVARCHAR(80),
    Information NVARCHAR(80)
)

In [ ]:
INSERT INTO SlackParams (Token_Name, Information)
VALUES ('SLACK_BG_TOKEN','tokenstring from api.Slack.com ')

# PostMessages

Post messages require 2 parametres to be passed to it.  Channel Name or Channel ID and the Message you want to send. The Channel parameter can either be the Channel Name of the Channel ID.  The Channel name is easier to identify,  however the PostMessage command in python returns the ChannelID.  If you are wanting to edit the message later this is useful to store and repost.  The Second required parameter is the Message.  This is text in Markdown language.

In [ ]:
CREATE OR ALTER PROCEDURE slackToChannel
@Channel NVARCHAR(200) = '#dba' OUTPUT,
@ThreadId NVARCHAR(200) = '' OUTPUT,
@Message NVARCHAR(MAX),
@Update bit = FALSE
AS
BEGIN
    DECLARE @PythonCode NVARCHAR(MAX)
    DECLARE @CertId NVARCHAR(80)
    DECLARE @ThreadInfo VARCHAR(500)

    SELECT @CertId = Information
    FROM master.dbo.SlackParams
    WHERE Token_Name = 'SLACK_BG_TOKEN'

    IF (@Update = 0) 
    BEGIN 
    --New Message
        IF @threadId IS NULL OR @threadId = ''
        BEGIN  
            SET @ThreadInfo = ''
        END
        ELSE    
        BEGIN 
            SET @threadInfo = ' , thread_ts = "' + @ThreadId + '"'
        END
        
            SET @PythonCode = N'import slack
# token is taken from the table Params  and entered in thepython script below.
client = slack.WebClient(token=''' + @CertId + ''')

#  post to channel submitted to the proc @channel 
MessageReturn = client.chat_postMessage(channel="' + @Channel + '", text="' + @Message + '"' + @threadInfo + ')

thread_ts = MessageReturn["ts"]
channelid = MessageReturn["channel"]
'

        EXECUTE sp_execute_external_script @language = N'Python',
                @script = @PythonCode,
                @params = N'@thread_ts VARCHAR(200) OUTPUT, @channelid VARCHAR(200) OUTPUT',
                @channelid = @Channel OUTPUT, @thread_ts = @threadId OUTPUT
    END
    ELSE
    BEGIN 
    --Update a Message
        IF @threadId IS NULL OR @threadId = ''
        BEGIN   
            PRINT 'Thread Id required to update a message!!'
        END
        ELSE
        BEGIN
            SET @threadInfo = ' , ts = "' + @threadId + '"'
            SET @PythonCode = N'import slack
client = slack.WebClient(token=''' + @CertId  + ''')

#update message posted to Channel
MessageReturn = client.chat_update(channel=''' + @Channel + ''', text="' + @Message + '"' + @threadInfo + ')

thread_ts = MessageReturn["ts"]
channelid = MessageReturn["channel"]'
            EXECUTE sp_execute_external_script @language = N'Python',
                @script = @PythonCode,
                @params = N'@thread_ts VARCHAR(200) OUTPUT, @channelid VARCHAR(200) OUTPUT',
                @channelid = @Channel OUTPUT, @thread_ts = @threadId OUTPUT


        END
    END
    print @PythonCode
END

In [ ]:
EXEC slackToChannel @Channel = 'sqlserverslack', @Message = 'Hello World'